In [1]:
import numpy as np
from scipy.linalg import inv, det
import warnings
from sklearn.covariance import LedoitWolf
from typing import Dict
# --- 1. 分布のパラメータ推定 ---

def estimate_single_gaussian_params(X_data: np.ndarray) -> Dict[str, np.ndarray]:
    """
    単一の高次元データセットから、多変量正規分布の平均ベクトルと共分散行列を推定する。
    サンプル数が次元数より少ない場合、Ledoit-Wolf収縮法を用いて共分散行列を頑健に推定する。

    Args:
        X_data (np.ndarray): 単一のクラスタ（またはデータセット）に属するデータ (N, D)。

    Returns:
        Dict: {'mu': 平均ベクトル, 'Sigma': 共分散行列}
    """
    N, D = X_data.shape # サンプル数 N, 次元数 D

    if N == 0:
        raise ValueError("Input data array must not be empty.")
    
    # 1. 平均ベクトル (mu) の推定
    mu = np.mean(X_data, axis=0)

    # 2. 共分散行列 (Sigma) の推定
    if N == 1:
        # サンプル数が1の場合、共分散はゼロ
        warnings.warn("N=1. Covariance matrix is set to zero (plus regularization).")
        Sigma = np.zeros((D, D))
        
    elif N < D + 1:
        # N < D+1 の場合: 特異行列になるリスクが高いため、Ledoit-Wolf収縮推定を使用
        warnings.warn(f"N={N} < D+1={D+1}. Using Ledoit-Wolf shrinkage for robust covariance estimation.")
        
        # Ledoit-Wolf収縮推定器を初期化・学習
        lw = LedoitWolf()
        lw.fit(X_data)
        Sigma = lw.covariance_
        
    else:
        # N >= D+1 の場合: 標準的な最尤推定を使用
        Sigma = np.cov(X_data, rowvar=False)

    # 3. 最終的な正則化チェック
    # 収縮推定を行ってもなお条件数が悪い場合に、微小な値を加算
    if np.linalg.cond(Sigma) > 1e15: # より厳しい条件数でチェック
        warnings.warn("Covariance matrix highly ill-conditioned. Applying final small regularization.")
        Sigma += np.eye(D) * 1e-6
        
    return {'mu': mu, 'Sigma': Sigma}

# --- 2. 類似度測定 (非類似度) ---

def kl_divergence_gaussian(mu1, Sigma1, mu2, Sigma2) -> float:
    """
    多変量ガウス分布 N1 から N2 へのKL情報量 D_KL(N1 || N2) を計算する。
    """
    D = mu1.shape[0]

    try:
        Sigma2_inv = inv(Sigma2)
    except np.linalg.LinAlgError:
        warnings.warn("Sigma2 is singular. KL divergence is undefined.")
        return np.nan

    # 1. ログデターミナント項
    log_det_term = np.log(det(Sigma2) / det(Sigma1))

    # 2. トレース項
    trace_term = np.trace(Sigma2_inv @ Sigma1)

    # 3. マハラノビス距離項 (平均の違い)
    diff_mu = mu2 - mu1
    mahalanobis_term = diff_mu.T @ Sigma2_inv @ diff_mu

    kl_div = 0.5 * (log_det_term + trace_term + mahalanobis_term - D)

    return kl_div

# --- 3. 類似度測定 (重なり) ---

def bhattacharyya_coefficient_gaussian(mu1, Sigma1, mu2, Sigma2) -> float:
    """
    多変量ガウス分布間のバタチャリヤ係数 BC を計算する。
    """
    D = mu1.shape[0]
    
    # 共分散行列の平均 Sigma = 0.5 * (Sigma1 + Sigma2)
    Sigma = 0.5 * (Sigma1 + Sigma2)

    try:
        Sigma_inv = inv(Sigma)
    except np.linalg.LinAlgError:
        warnings.warn("Sigma (mean covariance) is singular. Bhattacharyya Coefficient is undefined.")
        return np.nan

    # 1. バタチャリヤ距離の平均項（マハラノビス距離の変形）
    diff_mu = mu1 - mu2
    db_mu_term = 0.125 * diff_mu.T @ Sigma_inv @ diff_mu

    # 2. バタチャリヤ距離の共分散項
    db_cov_term = 0.5 * np.log(det(Sigma) / np.sqrt(det(Sigma1) * det(Sigma2)))

    # バタチャリヤ距離
    db_distance = db_mu_term + db_cov_term
    
    # バタチャリヤ係数 = exp(-距離)
    bc = np.exp(-db_distance)

    return bc

# --- 4. マハラノビス距離 (参考) ---

def mahalanobis_distance(mu1, mu2, Sigma_pooled) -> float:
    """
    プールされた共分散行列 Sigma_pooled を用いて、2つの平均間のマハラノビス距離を計算する。
    """
    try:
        Sigma_inv = inv(Sigma_pooled)
    except np.linalg.LinAlgError:
        warnings.warn("Sigma_pooled is singular. Mahalanobis distance is undefined.")
        return np.nan
    
    diff_mu = mu1 - mu2
    # 距離の2乗
    dist_sq = diff_mu.T @ Sigma_inv @ diff_mu
    
    return np.sqrt(dist_sq)





In [3]:
# --- サンプル実行 ---
np.random.seed(42)

# 高次元データ (D=10)
D = 10
N_A = 100
N_B = 120
N_C = 80

# サンプル用のパラメータ設定
mu_A = np.zeros(D)
Sigma_A = np.eye(D) * 2

mu_B = np.ones(D) * 1.5 # 少し離れた位置
Sigma_B = np.eye(D) * 1 # 分散が小さい（密）

mu_C = np.zeros(D) # Aと同じ位置だが形状が異なる
Sigma_C = np.diag(np.linspace(0.5, 3.0, D)) # 非常に細長い形状

# データ生成
X_A = np.random.multivariate_normal(mu_A, Sigma_A, N_A)
X_B = np.random.multivariate_normal(mu_B, Sigma_B, N_B)
X_C = np.random.multivariate_normal(mu_C, Sigma_C, N_C)

# 元データ X を作成 (N=300, D=10)
X = np.vstack([X_A, X_B, X_C])

# クラスタインデックスの辞書
cluster_indices = {
    'A': list(range(0, N_A)),
    'B': list(range(N_A, N_A + N_B)),
    'C': list(range(N_A + N_B, N_A + N_B + N_C))
}

## ステップ 1: 分布パラメータの推定
params = estimate_gaussian_params(X, cluster_indices)

print("## 📊 ステップ 1: 分布パラメータの推定結果")
print(f"クラスタ A: mu_norm={np.linalg.norm(params['A']['mu']):.2f}, det(Sigma)={det(params['A']['Sigma']):.2e}")
print(f"クラスタ B: mu_norm={np.linalg.norm(params['B']['mu']):.2f}, det(Sigma)={det(params['B']['Sigma']):.2e}")
print(f"クラスタ C: mu_norm={np.linalg.norm(params['C']['mu']):.2f}, det(Sigma)={det(params['C']['Sigma']):.2e}")

## ステップ 2: 類似度測定の実行 (AとB、AとCを比較)

# パラメータ抽出
mu_A, Sigma_A = params['A']['mu'], params['A']['Sigma']
mu_B, Sigma_B = params['B']['mu'], params['B']['Sigma']
mu_C, Sigma_C = params['C']['mu'], params['C']['Sigma']

print("\n## 📈 ステップ 2: クラスタ間類似度の計算")
print("--- A (分散大) vs B (分散小、位置遠い) ---")
kl_AB = kl_divergence_gaussian(mu_A, Sigma_A, mu_B, Sigma_B)
kl_BA = kl_divergence_gaussian(mu_B, Sigma_B, mu_A, Sigma_A)
bc_AB = bhattacharyya_coefficient_gaussian(mu_A, Sigma_A, mu_B, Sigma_B)

print(f"KL情報量 D_KL(A || B) (非類似度): {kl_AB:.4f}")
print(f"KL情報量 D_KL(B || A) (非類似度): {kl_BA:.4f} (非対称性の例)")
print(f"バタチャリヤ係数 BC(A, B) (重なり): {bc_AB:.4f} (1に近いほど類似)")

print("\n--- A (球状) vs C (細長い、位置近い) ---")
kl_AC = kl_divergence_gaussian(mu_A, Sigma_A, mu_C, Sigma_C)
kl_CA = kl_divergence_gaussian(mu_C, Sigma_C, mu_A, Sigma_A)
bc_AC = bhattacharyya_coefficient_gaussian(mu_A, Sigma_A, mu_C, Sigma_C)

print(f"KL情報量 D_KL(A || C): {kl_AC:.4f}")
print(f"KL情報量 D_KL(C || A): {kl_CA:.4f}")
print(f"バタチャリヤ係数 BC(A, C): {bc_AC:.4f}")

## 📊 ステップ 1: 分布パラメータの推定結果
クラスタ A: mu_norm=0.24, det(Sigma)=4.10e+02
クラスタ B: mu_norm=4.90, det(Sigma)=5.92e-01
クラスタ C: mu_norm=0.48, det(Sigma)=2.31e+01

## 📈 ステップ 2: クラスタ間類似度の計算
--- A (分散大) vs B (分散小、位置遠い) ---
KL情報量 D_KL(A || B) (非類似度): 16.2178
KL情報量 D_KL(B || A) (非類似度): 7.8807 (非対称性の例)
バタチャリヤ係数 BC(A, B) (重なり): 0.0884 (1に近いほど類似)

--- A (球状) vs C (細長い、位置近い) ---
KL情報量 D_KL(A || C): 2.7139
KL情報量 D_KL(C || A): 1.5463
バタチャリヤ係数 BC(A, C): 0.6450


# metric

In [2]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# medoid or centroid
def calculate_cluster_similarity(vecs_a, vecs_b, mode='medoid'):
    """
    2つのベクトル集合間の類似度/距離を計算する
    
    Parameters:
    vecs_a, vecs_b: np.array (n_samples, n_features)
    mode: 'medoid' (代表点同士), 'average' (全体の平均)
    
    Returns:
    dict: 類似度と距離
    """
    
    if mode == 'medoid':
        # 各集合内で「他の点との距離の総和が最小」な点をメドイドとする
        # 簡易的に中心点に最も近い実データを選択
        def get_medoid(vecs):
            centroid = np.mean(vecs, axis=0).reshape(1, -1)
            # 重心に最も近いデータのインデックスを探す
            sim = cosine_similarity(vecs, centroid)
            return vecs[np.argmax(sim)].reshape(1, -1)
        
        rep_a = get_medoid(vecs_a)
        rep_b = get_medoid(vecs_b)
    else:
        # 平均ベクトル（セントロイド）を使用
        rep_a = np.mean(vecs_a, axis=0).reshape(1, -1)
        rep_b = np.mean(vecs_b, axis=0).reshape(1, -1)

    # コサイン類似度を算出
    similarity = cosine_similarity(rep_a, rep_b)[0][0]
    
    # 距離に変換 (0.0: 同一 ～ 2.0: 正反対)
    distance = 1 - similarity

    # ユークリッド距離
    euclidean_distance = np.linalg.norm(rep_a - rep_b)
    
    return {
        "similarity": similarity,
        "distance": distance,
        "euclidean_distance": euclidean_distance
    }

In [ ]:
def calculate_cluster_centroid(vecs):
    """
    ベクトル集合のセントロイド（平均ベクトル）を計算する
    """
    return np.mean(vecs, axis=0)
def calculate_cluster_medoid(vecs):
    """
    ベクトル集合のメドイドを計算する
    """
    centroid = np.mean(vecs, axis=0).reshape(1, -1)
    sim = cosine_similarity(vecs, centroid)
    return vecs[np.argmax(sim)].reshape(1, -1)


# 実データ


In [11]:
import numpy as np
# ベクトル
vectors = np.load("../../d3-app/data/vector.npy")
# point_to_cluster_map
point_to_cluster_map = np.load("../../d3-app/data/point_cluster_map.npy")

# unique clusters
unique_clusters = np.unique(point_to_cluster_map)

# 低次元投影
projected_vectors = np.load("../../d3-app/data/projection.npy")

# leave IDs
leave_ids = np.load("../../d3-app/data/leaves.npy")

In [18]:
# cluster数
print(f"Number of unique clusters: {len(unique_clusters)}")
print(f"sample unique clusters: {unique_clusters[:10]}")

# point cluster map
print(f"Number of points: {len(point_to_cluster_map)}")
print(f"Sample point to cluster map: {point_to_cluster_map[:10]}")
# leave ids
print(f"Number of leave IDs: {len(leave_ids)}")
print(f"Sample leave IDs: {leave_ids[:10]}")

Number of unique clusters: 871
sample unique clusters: [115754 115755 115756 115757 115758 115759 115760 115761 115762 115763]
Number of points: 115754
Sample point to cluster map: [115756 115754 115754 115928 115754 115924 115756 115754 116203 115754]
Number of leave IDs: 443
Sample leave IDs: [115760 115763 115769 115771 115774 115779 115781 115783 115786 115788]


In [16]:
# 115760 is in unique clusters
print(115760 in unique_clusters)

True


In [23]:
# leaves idの点の数の分布(histogram), point_cluster_mapはクラスタidの列

leave_points = { cluster_id: np.where(point_to_cluster_map == cluster_id)[0] for cluster_id in leave_ids}

leave_sizes = { cluster_id: len(leave_points[cluster_id]) for cluster_id in leave_points}
import plotly.express as px
fig = px.histogram(x=list(leave_sizes.values()), nbins=30, title="Distribution of Leave Sizes")
fig.show()

# 0の数
num_zero_size = sum(1 for size in leave_sizes.values() if size == 0)
print(f"Number of leaves with size 0: {num_zero_size}")

# 最小サイズ
min_size = min(leave_sizes.values())
print(f"Minimum leave size: {min_size}")
# 最大サイズ
max_size = max(leave_sizes.values())
print(f"Maximum leave size: {max_size}")



Number of leaves with size 0: 0
Minimum leave size: 5
Maximum leave size: 176


## medoid, centroidのcontinity, trustworthiness

In [37]:
import numpy as np
from tqdm import tqdm
from sklearn.manifold import trustworthiness
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cluster_representative_metrics(X_high, X_low, point_to_cluster_map, leaves_ids, mode='medoid', k=5):
    """
    leaves_idsで指定された各クラスタを代表点(メドイド/セントロイド)に集約し、
    クラスタ間の配置維持率を計算する
    """
    
    rep_high_list = []
    rep_low_list = []
    valid_ids = []

    print(f"Calculating {mode}s for each cluster...")
    for cluster_id in leaves_ids:
        mask = (point_to_cluster_map == cluster_id)
        vecs_h = X_high[mask]
        vecs_l = X_low[mask]
        
        if len(vecs_h) == 0:
            continue
            
        if mode == 'medoid':
            # 高次元での平均を基準にメドイドのインデックスを特定
            centroid_h = np.mean(vecs_h, axis=0).reshape(1, -1)
            sim = cosine_similarity(vecs_h, centroid_h)
            medoid_idx = np.argmax(sim)
            
            rep_high_list.append(vecs_h[medoid_idx])
            rep_low_list.append(vecs_l[medoid_idx])
        else: # centroid
            rep_high_list.append(np.mean(vecs_h, axis=0))
            rep_low_list.append(np.mean(vecs_l, axis=0))
        
        valid_ids.append(cluster_id)

    # 代表点の行列を作成
    rep_high = np.array(rep_high_list)
    rep_low = np.array(rep_low_list)

    if len(rep_high) <= k:
        raise ValueError(f"クラスタ数が近傍数 k={k} 以下のため計算できません。")

    # 全体としての維持スコアを計算
    # Trustworthiness: 低次元の近傍が、高次元でも近傍か
    overall_t = trustworthiness(rep_high, rep_low, n_neighbors=k)
    # Continuity: 高次元の近傍が、低次元でも近傍か
    overall_c = trustworthiness(rep_low, rep_high, n_neighbors=k)

    return {
        "overall_trustworthiness": overall_t,
        "overall_continuity": overall_c,
        "cluster_count": len(valid_ids)
    }

# --- 実行例 ---
# results = calculate_cluster_representative_metrics(
#     vectors, 
#     vectors_low, 
#     point_to_cluster_map, 
#     leaves_ids=my_cluster_list, 
#     mode='medoid', 
#     k=10
# )
# print(results)

In [38]:
for k in [2, 3, 4, 5, 10, 15, 40, 100]:
    results = calculate_cluster_representative_metrics(
        vectors, 
        projected_vectors, 
        point_to_cluster_map, 
        leaves_ids=leave_ids, 
        mode='medoid', 
        k=k
    )
    print(f"param: k={k}")
    print(results)

for k in [2, 3, 4, 5, 10, 15, 40, 100]:
    results = calculate_cluster_representative_metrics(
        vectors, 
        projected_vectors, 
        point_to_cluster_map, 
        leaves_ids=leave_ids, 
        mode='average', 
        k=k
    )
    print(f"param: k={k}")
    print(results)

Calculating medoids for each cluster...
param: k=2
{'overall_trustworthiness': 0.7955762371050624, 'overall_continuity': 0.8835686972421462, 'cluster_count': 443}
Calculating medoids for each cluster...
param: k=3
{'overall_trustworthiness': 0.7842869462740207, 'overall_continuity': 0.872333371127397, 'cluster_count': 443}
Calculating medoids for each cluster...
param: k=4
{'overall_trustworthiness': 0.7827501234682822, 'overall_continuity': 0.8576649885323177, 'cluster_count': 443}
Calculating medoids for each cluster...
param: k=5
{'overall_trustworthiness': 0.778995874523235, 'overall_continuity': 0.8445281648114995, 'cluster_count': 443}
Calculating medoids for each cluster...
param: k=10
{'overall_trustworthiness': 0.7678333531345294, 'overall_continuity': 0.8067376869562921, 'cluster_count': 443}
Calculating medoids for each cluster...
param: k=15
{'overall_trustworthiness': 0.754054247733706, 'overall_continuity': 0.7778415564871547, 'cluster_count': 443}
Calculating medoids for

## 類似度を計算

In [ ]:
# 組み合わせごとに類似度を計算、辞書に保存

from itertools import combinations
similarity_results = {}
for cluster_a, cluster_b in combinations(unique_clusters, 2):
    # 各クラスタのベクトルを抽出
    vecs_a = vectors[point_to_cluster_map == cluster_a]
    vecs_b = vectors[point_to_cluster_map == cluster_b]
    
    # 類似度計算
    result = calculate_cluster_similarity(vecs_a, vecs_b, mode='medoid')
    similarity_results[(cluster_a, cluster_b)] = result["distance"]

# save
# np.save("src/experiments/21_cluster_similarity/cluster_similarity_distances.npy", similarity_results)
    

FileNotFoundError: [Errno 2] No such file or directory: 'src/experiments/21_cluster_similarity/cluster_similarity_distances.npy'